# Hands-on with the Turtlebot 3
## Simultaneous Localization and Mapping (SLAM)

The SLAM (Simultaneous Localization and Mapping) problem is the problem of building a map of the environment while simultaneously determining the robot’s position relative to this map. This problem is also referred to as Concurrent Mapping and Localization (CML). A mobile robot moves through an unknown environment, starting at a known location (often given by coordinates {0,0,0}). Its motion is uncertain due to the robot’s model and sensor noise.
As it roams, the robot can detect the environment using a form of observation measurement. To compute the map and the robot’s location within this map, the probability of accuracy of the data must be included. 

SLAM has received considerable attention since the 1990's. Several approaches have been proposed such as Kalman and Particle filter SLAM. Gmapping is arguably the most popular software implementation of SLAM. It employs a Rao-Blackwellized particle filter (RBPF) and builds a 2D map based on laser scan data. The code and documentation are available on the [OpenSLAM website](http://openslam.org/gmapping.html). It is very easy to integrate into ROS, and can even be run on a low cost robot-platform like the Turtlebot 3. This section will demonstrate how to start SLAM on this robot, how to visualize the data and how to record it for future use.

## Visualization in rviz

rviz is a 3D visualization tool for ROS. It is included in a full desktop install of ROS, but can of course be installed separately as well. rviz can display robot positions (with a complete robot model if it is provided in a URDF file), laser scans, point clouds and much information that is being published by ROS nodes. To start rviz, a roscore needs to be running:

In [ ]:
!roscore

In a separate terminal, you can start rviz:

In [ ]:
!rviz

You can use the left mouse button to rotate the camera, the right mouse button to zoom and the middle mouse button 
to pan. On the left hand side of the window the display options are listed. The global options are listed at the top. Parameters like background color and frame rate can be adjusted here. There is also an option called 'Fixed Frame', which referes to the static coordinate frame relative to which all other information is referenced. By default, the frame 'odom' is selected, but 'base_footprint', 'world' and 'map' are also often used. The fixed frame should not be moving (e.g. it should not be attached to a mobile robot), doing so might result in erroneous visualization. 

Below the global options, you can adjust the settings of the visualization grid, such as the cell size and the color. Below that, you will find the only topic that is shown by default, namely the /tf topic. tf is an abbreviation for transfer frame, and contains information about the relative location of all coordinate frames in the system. Eventually, all frames should be directly or indirectly referenced relative to the fixed frame, thus building a so called 'tf-tree'.

At the bottom left of the rviz window, the 'Add' button allows you to visualize additional topics. By selecting the LaserScan message type for example, an additional display option will appear. You can then set parameters like the laserscan topic and the size and color of the markers. You can add additional display options in a similar manner. You can even have two display options of the same type (for example, 2 camera displays). This option only makes sense when the data being displayed originates from different topics though.

Once you have added the desired display options, you can save your configuration as a .rviz file. Doing so will allow you to have the exact same visualization window in the future (and thus prevents having to add the same topics over and over again). To save your rviz file:

1. Select the 'file' option in the window bar of the rviz panel. 
2. Select 'save config as'. 
3. Choose an appropriate name and location for your file. Select the default location (/home/ROS-workshop/.rviz) and 'example' as a filename to follow along with the rest of the chapter.

You can now safely close rviz. To open the same configuration file in rviz, you can either type the ```rviz``` command in a terminal, then navigate to the file be selecting 'file' and then 'open'. Alternatively, you can pass the configuration file as an argument when opening rviz:

In [ ]:
!rviz rviz  -d $HOME/.rviz/example.rviz

The argument after ```-d``` should be the path to the rviz file. If everything works correctly, you should see the display options you added earlier in stead of the default window.

## Gmapping with the Turtlebot 3

We will now move on to a more practical example of SLAM in ROS. To this end, we will build a map using gmapping and a Turtlebot 3 robot. This robot has an onboad computer, namely a raspberry pi 3 (RPi). The RPi has sends commands to the motors and reads out the various sensors. It relays this information to a remote pc, which does the actual processing. The RPi's on our robots have been configured to function as an access point, which means that a connection from your computer to the robot can be established just like any other wifi-connection. There is no central connection point, and thus no central point of failure. 

Before we can move on to the actual mapping, a roscore needs to be started on the robot, which will listen to the roscore on your PC in a master/slave configuration. First start a roscore on your own pc, then:

1. Flip the switch at the back of the Turtlebot to turn it on.
2. On the remote PC (master) connect to the Turtlebot 3 wifi (password = groupRBP). It can take a few minutes for the robot to show in the available wireless connections.
3. Establish an SSH (secure shell) connection:
    1. Open a terminal and type: ssh rosberrypi@192.168.42.1
    2. When prompted for a password, type: *groupRBP*. Unlike most applications, you will not see any symbols while typing the password. However, the keys are still being registered. So type the password and hit enter.
    3. If you are asked to add rosberrypi to the list of known hosts, type 'yes' and press enter
    4. If everything went as expected, the user in the terminal should have changed. All your commands should now be preceded by *'rosberrypi@rosberrypi'*. Any command you now type in this terminal is executed on the raspberry pi.
4. Start a roscore on the pi with the following command: ```bringup```. Besides starting a regular roscore, communication with the LIDAR and motors is now initialized as well.
5. To test that everything is working correctly, open a new terminal and type the following command on your own PC:
```roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch```. To open terminal in a new tab of the same window, press CTRL+SHIFT+T.
6. You should now be able to drive the Turtlebot around with your keyboard. When you are done, end the process with CTRL+C. Leave the other two terminals open.

We are now ready to start gmapping. Open a new terminal and run:

In [ ]:
!roslaunch ros_workshop turtlebot_slam.launch

This will start gmapping and rviz. Once again, you can steer the turtlebot with your keyboard. The map will get updated as you drive. If you want to save the map then run:

In [ ]:
!rosrun map_server map_saver -f ~/map

This will save the map in your home directory with the filename 'map'. The map will be saved in both .pgm and .yaml format. 

## Rosbag files

One of the nice featurs of ROS is that subscribers do not care about who is publishing. This means that data can originate from a physical system (such as a robot) a simulation (such as Gazebo) or even from recorded experiments.  Rosbag is a a tool for recording and playing back data from a ROS system. A rosbag file is a specialized file format that contains the messages from one or several ROS topics. API's exist for use in the [command line](http://wiki.ros.org/rosbag/Commandline), [C++](http://wiki.ros.org/rosbag/Code%20API#cpp_api) and [python](http://wiki.ros.org/rosbag/Code%20API#py_api). The recording process is a node itself, which means that it can also be started from inside a launch file. The rest of this tutorial will focus on the command line interface. 

### Recording bag files

To record one or more topics currently being published, simply run rosbag record, for example the following command will record the '/tf' and '/cmd_vel' topics:

In [ ]:
!rosbag record tf cmd_vel

Recording all topics is accomplished with the -a argument:

In [ ]:
!rosbag record -a

The default name of the file is the current unix timestamp. To specify a different name, use the -O or -output name. The following command will create a bag file called chatter.bag with the messages from the /chatter topic.

In [ ]:
!rosbag record -O chatter.bag /chatter

### Playing bag files

Playing back recorded data is accomplished by using 'rosbag play'. A roscore already has to be running for this command to work. For example to play back data from a file called 'recorded1.bag' run:

In [ ]:
!rosbag play recorded1.bag

You can also get a summary of the contents of a bag file before actually playing back the data with 'rosbag info':

In [ ]:
!rosbag info recorded1.bag

### Turtlebot example
roscore (make sure both bashrc files have correct IP addresses)
SSH -> bringup

roslaunch turtlebot_slam record:=True



roslaunch turtlebot_slam play:=BAGFILE.bag

# Navigation stack

The [ROS navigation stack](http://wiki.ros.org/navigation) is a collection of packages that enable autonomous operation for ROS-enabled robots. Sensor data such as odometry and laser scans is taken as an input, and actuator commands are sent to a mobile robot. Succesfull use of the navigation stack depends on a number of hard and software requirements.

Software requirements:
 - The robot must be controlled via a ROS master
 - A [tf](http://wiki.ros.org/tf) transform tree must exist which describes the relation of the robot joints.
 - Sensor data must be published in the right message types.
 - The navigation stack must be configured according to the shape and dynamics of the robot.
 
 Hardware requirements:
- The stack is meant for both differential drive and holonomic wheeled robots only. It assumes that the mobile base is controlled by sending desired velocity commands to achieve in the form of: x velocity, y velocity, theta velocity.
- A planar laser mounted somewhere on the mobile base is required. This laser is used for map building and localization.
- The Navigation Stack was developed on a square robot, so its performance will be best on robots that are nearly square or circular. It does work on robots of arbitrary shapes and sizes, but it may have difficulty with large rectangular robots in narrow spaces like doorways.

A list of robots which use the navigation stack can be found on the [ROS wiki](http://wiki.ros.org/navigation/RobotsUsingNavStack). Fortunately, the Turtlebot 3 is on that list. As such we can use it to get some hands-on experience.
